# Sanity checks

In [ ]:
# Imports

import sys
sys.path.append('../..') # So we can import our modules 

import os
import pandas as pd

## Frame extraction

### Pornography-800

In [ ]:
data_loc_800 = "/nas-ctm01/datasets/public/BIOMETRICS/pornography-database/data-processed/middle_20"
os.path.isfile(f"{data_loc_800}/data.csv")

In [ ]:
frames_800 = pd.read_csv(f"{data_loc_800}/data.csv")
frames_800["video"] = [frame_name.split("#")[0] for frame_name in frames_800["frame"]]
frames_800

In [ ]:
agg = {"video": "first", "label": "first", "frame": "size"}
videos_800 = frames_800[["video", "frame", "label"]]
videos_800 = videos_800.groupby("video").aggregate(agg).reset_index(drop=True)
videos_800 = videos_800.rename(columns={"frame": "frame_count"})
videos_800

In [ ]:
# Videos from which it was not possible to extract 20 frames
videos_800[videos_800["frame_count"] < 20]

In [ ]:
len(videos_800) # 7 videos with 0 frames extracted

In [ ]:
len(frames_800[frames_800["label"] == 0]) # Number of non-porn frames

In [ ]:
len(frames_800[frames_800["label"] == 1]) # Number of porn frames

### Pornography-2k

In [ ]:
data_loc_2k = "/nas-ctm01/datasets/public/BIOMETRICS/pornography-2k-db/data-processed/middle_20"
os.path.isfile(f"{data_loc_2k}/data.csv")

In [ ]:
frames_2k = pd.read_csv(f"{data_loc_2k}/data.csv")
frames_2k["video"] = [frame_name.split("#")[0] for frame_name in frames_2k["frame"]]
frames_2k

In [ ]:
agg = {"video": "first", "label": "first", "frame": "size"}
videos_2k = frames_2k[["video", "frame", "label"]]
videos_2k = videos_2k.groupby("video").aggregate(agg).reset_index(drop=True)
videos_2k = videos_2k.rename(columns={"frame": "frame_count"})
videos_2k

In [ ]:
# Videos from which it was not possible to extract 40 frames
videos_2k[videos_2k["frame_count"] < 40]

In [ ]:
# Videos from which it was extracted more than 40 frames
videos_2k[videos_2k["frame_count"] > 40]

In [ ]:
len(frames_2k[frames_2k["label"] == 0]) # Number of non-porn frames

In [ ]:
len(frames_2k[frames_2k["label"] == 1]) # Number of porn frames

## Model training and testing

In [47]:
# TODO: change variables if running remotely
DATA_LOC = "../../data"
MODEL_NAME = "resnet50"
model_path = f"../../models/model_{MODEL_NAME}_freeze_False_epochs_10_batch_32_optim_sgd_optimized_False.pth"
results_path = f"../../results/model_{MODEL_NAME}_freeze_False_epochs_10_batch_32_optim_sgd_optimized_False.csv"

In [48]:
df_results = pd.read_csv(results_path)
df_results

,Frame,Target,Prediction,Confidence
0,vPorn000601#0.jpg,1,1,1.0
1,vPorn000601#1.jpg,1,1,1.0
2,vPorn000601#2.jpg,1,1,1.0
3,vPorn000601#3.jpg,1,1,1.0
4,vPorn000601#4.jpg,1,1,1.0
...,...,...,...,...
7995,vNonPorn000033#15.jpg,0,0,1.0
7996,vNonPorn000033#16.jpg,0,0,1.0
7997,vNonPorn000033#17.jpg,0,0,1.0
7998,vNonPorn000033#18.jpg,0,0,1.0


In [49]:
print("Videos correctly classified as pornographic:", len(df_results[(df_results["Target"] == 1) & (df_results["Prediction"] == 1)]))
print("Videos incorrectly classified as pornographic", len(df_results[(df_results["Target"] == 0) & (df_results["Prediction"] == 1)]))
print("Videos correctly classified as non-pornographic", len(df_results[(df_results["Target"] == 0) & (df_results["Prediction"] == 0)]))
print("Videos incorrectly classified as pornographic", len(df_results[(df_results["Target"] == 1) & (df_results["Prediction"] == 0)]))

Videos correctly classified as pornographic: 3586
Videos incorrectly classified as pornographic 377
Videos correctly classified as non-pornographic 3603
Videos incorrectly classified as pornographic 434


In [50]:
df_results["Video"] = df_results["Frame"].str.split("#").str[0]
df_results

,Frame,Target,Prediction,Confidence,Video
0,vPorn000601#0.jpg,1,1,1.0,vPorn000601
1,vPorn000601#1.jpg,1,1,1.0,vPorn000601
2,vPorn000601#2.jpg,1,1,1.0,vPorn000601
3,vPorn000601#3.jpg,1,1,1.0,vPorn000601
4,vPorn000601#4.jpg,1,1,1.0,vPorn000601
...,...,...,...,...,...
7995,vNonPorn000033#15.jpg,0,0,1.0,vNonPorn000033
7996,vNonPorn000033#16.jpg,0,0,1.0,vNonPorn000033
7997,vNonPorn000033#17.jpg,0,0,1.0,vNonPorn000033
7998,vNonPorn000033#18.jpg,0,0,1.0,vNonPorn000033


In [51]:
df_results[df_results["Frame"] == "vPorn000885#17.jpg"]

,Frame,Target,Prediction,Confidence,Video
5297,vPorn000885#17.jpg,1,0,0.938795,vPorn000885


In [52]:
def separate_frames(row):
    target = row["Target"]

    correct_frames, incorrect_frames, correct_confidences, incorrect_confidences = [], [], [], []

    for frame, prediction, confidence in zip(row['Frame'], row['Prediction'], row['Confidence']):
        if target == prediction: # Correct prediction
            correct_frames.append(frame)
            correct_confidences.append(confidence)
        else: # Incorrect prediction
            incorrect_frames.append(frame)
            incorrect_confidences.append(confidence)

    return pd.Series(
        [correct_frames, incorrect_frames, correct_confidences, incorrect_confidences], 
        index=['Correct_frames', 'Incorrect_frames', 'Correct_confidences', 'Incorrect_confidences']
    )

# Group by video and aggregate columns
grouped_df = df_results.groupby('Video').agg({
    'Target': 'first',
    'Prediction': lambda x: list(x),
    'Frame': lambda x: list(x),
    'Confidence': lambda x: list(x)
}).reset_index()

# Apply aggregation function
grouped_df[['Correct_frames', 'Incorrect_frames', 'Correct_confidences', 'Incorrect_confidences']] = grouped_df.apply(separate_frames, axis=1)

# Drop unnecessary columns
grouped_df.drop(["Frame", "Prediction", "Confidence"], axis=1, inplace=True)

grouped_df

,Video,Target,Correct_frames,Incorrect_frames,Correct_confidences,Incorrect_confidences
0,vNonPorn000024,0,"[vNonPorn000024#0.jpg, vNonPorn000024#1.jpg, v...",[],"[0.9999995, 0.9999989, 0.9999993, 0.9999994, 0...",[]
1,vNonPorn000030,0,"[vNonPorn000030#0.jpg, vNonPorn000030#1.jpg, v...",[],"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",[]
2,vNonPorn000031,0,[],"[vNonPorn000031#0.jpg, vNonPorn000031#1.jpg, v...",[],"[0.99740297, 0.99701476, 0.99594986, 0.9975376..."
3,vNonPorn000033,0,"[vNonPorn000033#0.jpg, vNonPorn000033#1.jpg, v...",[],"[0.9999664, 0.9998373, 0.9999877, 0.9999677, 0...",[]
4,vNonPorn000045,0,"[vNonPorn000045#0.jpg, vNonPorn000045#1.jpg, v...",[],"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",[]
...,...,...,...,...,...,...
395,vPorn000967,1,[],"[vPorn000967#0.jpg, vPorn000967#1.jpg, vPorn00...",[],"[0.9999875, 0.999992, 0.999998, 0.9999981, 0.9..."
396,vPorn000979,1,"[vPorn000979#0.jpg, vPorn000979#1.jpg, vPorn00...",[],"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",[]
397,vPorn000982,1,"[vPorn000982#0.jpg, vPorn000982#1.jpg, vPorn00...",[],"[0.9999666, 0.99999547, 0.9999995, 0.999997, 0...",[]
398,vPorn000988,1,"[vPorn000988#0.jpg, vPorn000988#1.jpg, vPorn00...",[],"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",[]


In [53]:
# Videos with both correctly and incorrectly classified frames
grouped_df[grouped_df.apply(lambda row: len(row["Correct_frames"]) > 0 and len(row["Incorrect_frames"]) > 0, axis=1)]

,Video,Target,Correct_frames,Incorrect_frames,Correct_confidences,Incorrect_confidences
14,vNonPorn000074,0,"[vNonPorn000074#0.jpg, vNonPorn000074#1.jpg, v...","[vNonPorn000074#4.jpg, vNonPorn000074#5.jpg, v...","[0.9919309, 0.9961201, 0.99666435, 0.7240497, ...","[0.65907955, 0.6168413, 0.78513324, 0.80699664..."
24,vNonPorn000125,0,"[vNonPorn000125#1.jpg, vNonPorn000125#5.jpg, v...","[vNonPorn000125#0.jpg, vNonPorn000125#2.jpg, v...","[0.91928047, 0.9764156, 0.99698395, 0.5928837,...","[0.7616825, 0.99560404, 0.9779554, 0.986406]"
41,vNonPorn000221,0,"[vNonPorn000221#0.jpg, vNonPorn000221#1.jpg, v...","[vNonPorn000221#5.jpg, vNonPorn000221#7.jpg, v...","[0.9999629, 0.99992025, 0.9968988, 0.99906296,...","[0.84622526, 0.70124227, 0.94930094, 0.9811663..."
50,vNonPorn000262,0,"[vNonPorn000262#0.jpg, vNonPorn000262#1.jpg, v...","[vNonPorn000262#3.jpg, vNonPorn000262#4.jpg, v...","[0.9296241, 0.55864614, 0.7481108]","[0.87332416, 0.96359754, 0.9834051, 0.97883135..."
58,vNonPorn000301,0,"[vNonPorn000301#0.jpg, vNonPorn000301#1.jpg, v...","[vNonPorn000301#9.jpg, vNonPorn000301#10.jpg, ...","[0.99972063, 0.9989241, 0.9998915, 0.9994648, ...","[0.5828616, 0.9511455, 0.96496856, 0.96458524,..."
61,vNonPorn000307,0,"[vNonPorn000307#0.jpg, vNonPorn000307#1.jpg, v...","[vNonPorn000307#4.jpg, vNonPorn000307#8.jpg, v...","[0.5312489, 0.5466, 0.5677561, 0.5028091, 0.51...","[0.52608895, 0.65619195, 0.69408935, 0.9878293..."
72,vNonPorn000354,0,"[vNonPorn000354#1.jpg, vNonPorn000354#2.jpg, v...","[vNonPorn000354#0.jpg, vNonPorn000354#4.jpg, v...","[0.9566756, 0.7111608, 0.5304088, 0.56245756, ...","[0.5882412, 0.7161426, 0.778778, 0.94857657, 0..."
79,vNonPorn000383,0,"[vNonPorn000383#0.jpg, vNonPorn000383#1.jpg, v...","[vNonPorn000383#8.jpg, vNonPorn000383#9.jpg]","[0.9972076, 0.99773216, 0.984125, 0.9520084, 0...","[0.9814895, 0.9976947]"
81,vNonPorn000394,0,[vNonPorn000394#13.jpg],"[vNonPorn000394#0.jpg, vNonPorn000394#1.jpg, v...",[0.52773774],"[0.98592806, 0.9929395, 0.99185187, 0.9843279,..."
88,vNonPorn000430,0,"[vNonPorn000430#1.jpg, vNonPorn000430#2.jpg, v...",[vNonPorn000430#0.jpg],"[0.9509108, 0.9989869, 0.99475217, 0.9972319, ...",[0.5706103]


In [54]:
# Videos with only incorrectly classified frames
grouped_df[grouped_df.apply(lambda row: len(row["Correct_frames"]) == 0 and len(row["Incorrect_frames"]) > 0, axis=1)]

,Video,Target,Correct_frames,Incorrect_frames,Correct_confidences,Incorrect_confidences
2,vNonPorn000031,0,[],"[vNonPorn000031#0.jpg, vNonPorn000031#1.jpg, v...",[],"[0.99740297, 0.99701476, 0.99594986, 0.9975376..."
55,vNonPorn000290,0,[],"[vNonPorn000290#0.jpg, vNonPorn000290#1.jpg, v...",[],"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
59,vNonPorn000304,0,[],"[vNonPorn000304#0.jpg, vNonPorn000304#1.jpg, v...",[],"[0.9977337, 0.99953353, 0.9994492, 0.9993524, ..."
67,vNonPorn000343,0,[],"[vNonPorn000343#0.jpg, vNonPorn000343#1.jpg, v...",[],"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
68,vNonPorn000345,0,[],"[vNonPorn000345#0.jpg, vNonPorn000345#1.jpg, v...",[],"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
70,vNonPorn000352,0,[],"[vNonPorn000352#0.jpg, vNonPorn000352#1.jpg, v...",[],"[0.9854938, 0.9980704, 0.9997373, 0.9998957, 0..."
75,vNonPorn000367,0,[],"[vNonPorn000367#0.jpg, vNonPorn000367#1.jpg, v...",[],"[0.9999995, 0.99999976, 0.99999964, 0.9999995,..."
82,vNonPorn000395,0,[],"[vNonPorn000395#0.jpg, vNonPorn000395#1.jpg, v...",[],"[0.97342664, 0.96763927, 0.9733001, 0.980342, ..."
166,vNonPorn000808,0,[],"[vNonPorn000808#0.jpg, vNonPorn000808#1.jpg, v...",[],"[0.9989249, 0.9986197, 0.9976025, 0.9969367, 0..."
192,vNonPorn000966,0,[],"[vNonPorn000966#0.jpg, vNonPorn000966#1.jpg, v...",[],"[0.9997745, 0.9997638, 0.8995691, 0.8633119, 0..."


In [55]:
# Videos with only correctly classified frames
grouped_df[grouped_df.apply(lambda row: len(row["Incorrect_frames"]) == 0 and row["Target"] == 1, axis=1)]

,Video,Target,Correct_frames,Incorrect_frames,Correct_confidences,Incorrect_confidences
199,vPorn000005,1,"[vPorn000005#0.jpg, vPorn000005#1.jpg, vPorn00...",[],"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",[]
201,vPorn000034,1,"[vPorn000034#0.jpg, vPorn000034#1.jpg, vPorn00...",[],"[0.9853506, 0.9880782, 0.9928063, 0.9779012, 0...",[]
202,vPorn000037,1,"[vPorn000037#0.jpg, vPorn000037#1.jpg, vPorn00...",[],"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",[]
203,vPorn000042,1,"[vPorn000042#0.jpg, vPorn000042#1.jpg, vPorn00...",[],"[1.0, 1.0, 0.9999999, 1.0, 1.0, 1.0, 1.0, 1.0,...",[]
205,vPorn000055,1,"[vPorn000055#0.jpg, vPorn000055#1.jpg, vPorn00...",[],"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",[]
...,...,...,...,...,...,...
394,vPorn000966,1,"[vPorn000966#0.jpg, vPorn000966#1.jpg, vPorn00...",[],"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",[]
396,vPorn000979,1,"[vPorn000979#0.jpg, vPorn000979#1.jpg, vPorn00...",[],"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",[]
397,vPorn000982,1,"[vPorn000982#0.jpg, vPorn000982#1.jpg, vPorn00...",[],"[0.9999666, 0.99999547, 0.9999995, 0.999997, 0...",[]
398,vPorn000988,1,"[vPorn000988#0.jpg, vPorn000988#1.jpg, vPorn00...",[],"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",[]


In [56]:
# Videos which would have been correctly classified if in a majority voting scheme
print(f"Videos with more correctly than incorrectly classified frames: {len(grouped_df[grouped_df.apply(lambda row: len(row['Correct_frames']) > len(row['Incorrect_frames']), axis=1)])} / {len(grouped_df)}")

Videos with more correctly than incorrectly classified frames: 358 / 400


In [59]:
df_results[df_results["Frame"] == "vPorn000885#17.jpg"]

,Frame,Target,Prediction,Confidence,Video
5297,vPorn000885#17.jpg,1,0,0.938795,vPorn000885


- vPorn000885#17 (5297) - really interesting example because when predicting porn, the model looks at the logo, but to predict non-porn the model looks at the sexual organ
- vPorn000676#9 (1789), vPorn000687#14 (3434) - interesting to see how the model is paying attention to the logo
- vNonPorn000367#3 (2383) - when it classifies a non-porn as porn, it is usually paying attention to actually relevant stuff
- vNonPorn000808#0 (4100) - here it was paying attention to the hands (skin)
- vNonPorn000966#0 (1520) - here it payed attention two the pigs, it might have been because of the shape or the color, which is similar to that of skin
- vPorn000005#0 (4140), vPorn000034#6 (686) - here, for example, although there is a logo, it payed attention to the skin areas